In [46]:
# Retrieve pickled dataframe with symbols' OHLCV
source_filename = "df_OHLCV_2025-03-03_clean.pkl"
destination_filename = "df_perf_ratios.pkl"

# days used in calculating ratios
days_ratio = [30, 63, 125, 250]

# risk free rate
risk_free_rate = 0.04

In [47]:
import os

# Create temp directory if it doesn't exist
temp_dir = os.path.join(os.getcwd(), 'temp')
os.makedirs(temp_dir, exist_ok=True)

# Get source and destination paths
source_path = os.path.join(temp_dir, source_filename)
dest_path = os.path.join(temp_dir, destination_filename)

print(f'temp_dir: {temp_dir}')
print(f'source_path: {source_path}')
print(f'dest_path: {dest_path}')


temp_dir: c:\Users\ping\Files_win10\python\py310\stocks\temp
source_path: c:\Users\ping\Files_win10\python\py310\stocks\temp\df_OHLCV_2025-03-03_clean.pkl
dest_path: c:\Users\ping\Files_win10\python\py310\stocks\temp\df_perf_ratios.pkl


In [48]:
import pandas as pd

# Load the DataFrame from the pickle file
df = pd.read_pickle(source_path)

# Display the first few rows of the DataFrame to verify
df = df[['Adj Close']].copy()
df

Adj Close
Symbol Date                 
AAPL   2025-03-03     238.03
       2025-02-28     241.84
       2025-02-27     237.30
       2025-02-26     240.36
       2025-02-25     247.04
...                      ...
FCOM   2024-03-08      47.87
       2024-03-07      47.85
       2024-03-06      47.13
       2024-03-05      47.21
       2024-03-04      47.47

[619500 rows x 1 columns]

In [49]:
import utils

list_dfs = utils.get_latest_dfs(df, days_ratio)

In [50]:
all_results = {}

for _df in list_dfs:
    tickers_in_df = _df.index.get_level_values(0).unique()
    for ticker in tickers_in_df:
        result_df = utils.analyze_stock(_df, ticker, risk_free_rate=risk_free_rate)
        if result_df is not None:
            # Extract the ticker name from the result_df index
            ticker_name = result_df.index[0]
            # Convert the single-row DataFrame to a dictionary
            metrics = result_df.iloc[0].to_dict()
            
            # Update the existing ticker entry or create a new one
            if ticker_name in all_results:
                all_results[ticker_name].update(metrics)
            else:
                all_results[ticker_name] = metrics

if all_results:
    combined_df = pd.DataFrame.from_dict(all_results, orient='index')
    print("\nCombined performance metrics DataFrame:")
    print(combined_df)
else:
    print("No performance metrics were calculated.")


Combined performance metrics DataFrame:
      Sharpe 30d  Sortino 30d  Omega 30d  Sharpe 63d  Sortino 63d  Omega 63d  \
A      -5.141319    -5.874034   0.387854   -1.557570    -2.072968   0.764167   
AA     -4.138950    -4.757844   0.486068   -3.534636    -4.033184   0.538406   
AAL    -6.545431    -6.369321   0.275131   -0.309140    -0.519767   0.936498   
AAON   -5.192318    -5.220528   0.288863   -3.497344    -3.650716   0.442304   
AAP    -2.981175    -3.289828   0.528001   -1.346271    -1.597772   0.775338   
...          ...          ...        ...         ...          ...        ...   
ZM     -0.949757    -1.295046   0.814267   -1.609598    -2.139231   0.735023   
ZS      0.217949     0.340288   1.037328   -0.797255    -1.081709   0.878738   
ZTO     1.049341     1.471367   1.172388    0.437474     0.613256   1.071279   
ZTS     0.904212     1.190219   1.167916   -0.577078    -0.736313   0.909223   
ZWS    -3.867918    -4.316013   0.522642   -2.628016    -3.051696   0.651794   

In [51]:
# Save the combined DataFrame to pickle file
combined_df.to_pickle(dest_path)
print(f"DataFrame successfully saved to {dest_path}")

DataFrame successfully saved to c:\Users\ping\Files_win10\python\py310\stocks\temp\df_perf_ratios.pkl
